## **Business Problem**

This data-driven project aims to derive actionable insights to enhance customer retention by focusing on high-value customers and identifying churn risks. The primary concern is the alarming customer churn rate faced by the company, leading to significant revenue loss. A higher churn rate indicates a critical issue that requires an in-depth analysis to uncover the underlying causes and patterns.

To better understand customer churn in the Telecom Company, a detailed analytical approach will be used.

First, a comprehensive examination of the dataset will shed light on factors such as customer demographics, service tenure, billing patterns, common subscriptions, preferred offers, and additional services demanded. Along with this, data visualization will play a key role. By using various graphs and charts, we can better understand the reasons behind the higher churn rates. These visualizations will meticulously highlight trends within the dataset for different customer segments, helping stakeholders to better understand customer behavior.

Lastly, to ensure that our findings are grounded in solid evidence, we will apply statistical tools, including t-tests and regression analyses, to validate our insights and further explore the factors driving customer churn.